- In Python eveything is an object. Every object has an `id`, a value and a `type`.
- Also, since everything is object,

 - we can assign it to a variable
 - we can pass it to a function
 - it can be returned by a function
 - we can add attributes to it
 - we can copy it
 
- Every object is either a class or an instance of a class. 
- A metaclass is the class of a class, i.e. a class is an instance of a metaclass. 
- Alternatively, every object in Python is an instance of a class. 
- `__bases__` attribute is used to find the classes an object inherits from. 
- `__class__` attribute is used to find the type of an object. 
- `__dict__` attribute is used to find all the attributes/methods of an object.

 
- In addition to checking the type of an object, `type` also has a completely  different ability. It can also create classes on the fly. `type` can take the description of a class as parameters, and return a class. 

In [17]:
object.__dict__

mappingproxy({'__class__': <attribute '__class__' of 'object' objects>,
              '__delattr__': <slot wrapper '__delattr__' of 'object' objects>,
              '__dir__': <method '__dir__' of 'object' objects>,
              '__doc__': 'The most base type',
              '__eq__': <slot wrapper '__eq__' of 'object' objects>,
              '__format__': <method '__format__' of 'object' objects>,
              '__ge__': <slot wrapper '__ge__' of 'object' objects>,
              '__getattribute__': <slot wrapper '__getattribute__' of 'object' objects>,
              '__gt__': <slot wrapper '__gt__' of 'object' objects>,
              '__hash__': <slot wrapper '__hash__' of 'object' objects>,
              '__init__': <slot wrapper '__init__' of 'object' objects>,
              '__init_subclass__': <method '__init_subclass__' of 'object' objects>,
              '__le__': <slot wrapper '__le__' of 'object' objects>,
              '__lt__': <slot wrapper '__lt__' of 'object' objects>,
 

In [24]:
print(type(object))
print(object.__bases__)
print(object.__class__)

<class 'type'>
()
<class 'type'>


In [25]:
class Test():
    pass

def echo(o):
    print(o)
    
print(Test)    

<class '__main__.Test'>


In [26]:
echo(Test)

<class '__main__.Test'>


In [27]:
hasattr(Test, 'some_attr')

False

In [28]:
Test.some_attr = 1
hasattr(Test, 'some_attr')

True

In [29]:
Test1 = Test
Test1.some_attr

1

Moreover, every object in Python is either an instance of a class or a class itself. For example 

In [19]:
a = 1
isinstance(a, int)

True

In [12]:
a = int()
print(a)
print(type(a))
print(type(int))
print(isinstance(a, int))

0
<class 'int'>
<class 'type'>
True


In [14]:
c = None
type(c), type(type(c))

(NoneType, type)

In [15]:
l = [1,2]
type(l), type(type(l))

(list, type)

In [5]:
int.__bases__, int.__class__

((object,), type)

In [8]:
b = float()
print(b)
print(type(b))

0.0
<class 'float'>


In [10]:
def test():
    pass

print(type(test))

<class 'function'>


In [11]:
print(type(type(test)))

<class 'type'>


In [2]:
class Foo():
    pass

foo = Foo()
print(type(foo))
print(foo.__class__)
print(type(Foo))
print(Foo.__class__)
print(Foo.__bases__)

<class '__main__.Foo'>
<class '__main__.Foo'>
<class 'type'>
<class 'type'>
(<class 'object'>,)


In [3]:
print(type(type))
print(type.__class__)
print(type.__class__.__class__)
print(type.__bases__)

<class 'type'>
<class 'type'>
<class 'type'>
(<class 'object'>,)


In [3]:
isinstance(foo, Foo), isinstance(Foo, type)

(True, True)

In [4]:
type.__bases__

(object,)

In [5]:
Foo.__bases__

(object,)

In [6]:
object.__bases__

()

Since `type` is in itself a class, we can subclass it to create a new `type`. Below we created a new `type` called `dd` by subclassing `type`. Now we can create classes from this newly created type `dd`. See below -

In [7]:
class dd(type):
    pass

class Complex(metaclass = dd):
    pass

In [8]:
print(type(Complex))
print(type(dd))

<class '__main__.dd'>
<class 'type'>


In [9]:
type(dd) is type(type)


True

In [10]:
isinstance(Complex, dd), isinstance(dd, type), isinstance(type, type), isinstance(Complex, type)

(True, True, True, True)

In [11]:
dd == type

False

In [12]:
Complex.__class__

__main__.dd

In [13]:
class Baz(metaclass = type):
    pass

In [14]:
Baz.__class__

type

When we created class `Foo` above, it implicitly became an instance of class `type`. However class `Complex` was created from `dd` type. See what happened in the case of `Baz` class.  

The fact that classes are instances of a class `type` allows us to program metaclasses. We can create classes, which inherit from the class `type`. So, a metaclass is a subclass of the class `type`.

We can subclass `type` by calling `type` with three parameters:

```
type(classname, superclasses, attributes_dict)
```

If type is called with three arguments, it will return a new type object. This provides us with a dynamic form of the class statement.

 - "classname" is a string defining the class name and becomes the name attribute;
 - "superclasses" is a list or tuple with the superclasses of our class. This list or tuple will become the bases attribute;
 - the attributes_dict is a dictionary, functioning as the namespace of our class. It contains the definitions for the class body and it becomes the dict attribute.


In [15]:
AA = type('AA',(),{})
AA

__main__.AA

In [16]:
x = AA()
x

In [17]:
type(AA), AA.__class__

(type, type)

In [18]:
class Robot:
    counter = 0
    def __init__(self, name):
        self.name = name
    def sayHello(self):
        return "Hi, I am " + self.name
    
def Rob_init(self, name):
    self.name = name

Robot2 = type("Robot2",(), {"counter":0, "__init__": Rob_init, "sayHello": lambda self: "Hi, I am " + self.name})

x = Robot2("Marvin")

print(x.name)
print(x.sayHello())

y = Robot("Marvin")

print(y.name)
print(y.sayHello())

print(x.__dict__)
print(y.__dict__)

Marvin
Hi, I am Marvin
Marvin
Hi, I am Marvin
{'name': 'Marvin'}
{'name': 'Marvin'}


In [24]:
dd = type(a)
dd

str

In [25]:
class dd(type):
    pass

class Complex(metaclass = dd):
    pass

In [26]:
ee = Complex()
ee

In [27]:
ee.__class__.__class__

__main__.dd

In [28]:
Complex.__class__

__main__.dd

In [29]:
print(type(Complex))

<class '__main__.dd'>


In [30]:
type(ee)

__main__.Complex

In [31]:
type(dd)

type

In [32]:
Foo

__main__.Foo

In [33]:
class ABC(Foo):
    pass

xyz = ABC()
xyz

In [34]:
isinstance(xyz, Foo)

True

In [35]:
a = 5
a.__class__

int

In [36]:
a.__class__.__class__

type

In [1]:
a =1
a.__class__

int

In [2]:
type(a).__name__

'int'

In [3]:
class Test:
    pass

print(Test.__class__)

<class 'type'>
